In [1]:
!git clone -b roshan_b https://github.com/roshant2003/Railway-Track-Fault-Detection-Project
!cp -r Railway-Track-Fault-Detection-Project/Railway/ /content/Dataset
!cp -r Railway-Track-Fault-Detection-Project/Railway_pre/ /content/Dataset_Pre
!pip install tensorflow==2.15

Cloning into 'Railway-Track-Fault-Detection-Project'...
remote: Enumerating objects: 10334, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10334 (delta 1), reused 11 (delta 1), pack-reused 10322 (from 1)
Receiving objects: 100% (10334/10334), 599.19 MiB | 17.55 MiB/s, done.
Resolving deltas: 100% (1/1), done.
Updating files: 100% (10309/10309), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.16.0
    Uninstalling wrapt-1.16.0:
   

In [2]:
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [3]:
data_dir = 'Dataset_Pre'

n_classes = 7

# Image parameters
n_channels = 3
image_size = 224

# Model parameters
pooling_avg = 'avg'
dense_layer_activation = 'softmax'
objective_function = 'categorical_crossentropy'

# Training parameters
loss_metrics = ['accuracy']
n_epochs = 100
early_stop_patience = 10

steps_per_epoch_training = 10
steps_per_epoch_validation = 10

batch_size_training = 100
batch_size_validation = 100

batch_size_testing = 1


In [4]:
# define generators
datagen = ImageDataGenerator(
        zoom_range=0.15,
        shear_range=0.15,
        horizontal_flip=True,
        validation_split=0.2,
        rescale=1.0/255)

train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size_training,
        class_mode='categorical',
        subset='training')

validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=(image_size, image_size),
        batch_size=batch_size_validation,
        class_mode='categorical',
        subset='validation')


Found 4125 images belonging to 7 classes.
Found 1028 images belonging to 7 classes.


In [5]:
# Define MobileNet Model
model = Sequential()
model.add(MobileNet(input_shape=(224, 224, 3), include_top=False, pooling=pooling_avg, weights='imagenet'))
model.add(Dense(n_classes, activation=dense_layer_activation))

# Freeze base layers
model.layers[0].trainable = False

model.summary()

from tensorflow.keras import optimizers

# Compile model
sgd = optimizers.SGD(learning_rate=0.01, weight_decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss=objective_function, metrics=loss_metrics)

from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

# Prepare early stopping to avoid overfitting
cb_early_stopper = EarlyStopping(monitor='val_loss', patience=early_stop_patience)
cb_checkpointer = ModelCheckpoint(filepath='../working/best_mobilenet', monitor='val_loss', save_best_only=True, mode='auto')

fit_history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch_training,
        epochs=n_epochs,
        validation_data=validation_generator,
        validation_steps=steps_per_epoch_validation,
        callbacks=[cb_checkpointer, cb_early_stopper]
)

model.load_weights("../working/best_mobilenet")

model.save("MobileNet_RTFD.keras")

17225924/17225924 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functi  (None, 1024)              3228864   
 onal)                                                           
                                                                 
 dense (Dense)               (None, 7)                 7175      
                                                                 
Total params: 3236039 (12.34 MB)
Trainable params: 7175 (28.03 KB)
Non-trainable params: 3228864 (12.32 MB)
_________________________________________________________________
Epoch 1/100
10/10 [==============================] - ETA: 0s - loss: 1.6249 - accuracy: 0.5000

10/10 [==============================] - 52s 5s/step - loss: 1.6249 - accuracy: 0.5000 - val_loss: 1.1536 - val_accuracy: 0.5780
Epoch 2/100
10/10 [==============================] - ETA: 0s - loss: 0.6040 - accuracy: 0.7924

10/10 [==============================] - 43s 4s/step - loss: 0.6040 - accuracy: 0.7924 - val_loss: 0.6219 - val_accuracy: 0.7490
Epoch 3/100
10/10 [==============================] - 33s 3s/step - loss: 0.4721 - accuracy: 0.8389 - val_loss: 0.8533 - val_accuracy: 0.6810
Epoch 4/100
10/10 [==============================] - 29s 3s/step - loss: 0.3765 - accuracy: 0.8730 - val_loss: 0.8224 - val_accuracy: 0.6930
Epoch 5/100
10/10 [==============================] - 34s 4s/step - loss: 0.3658 - accuracy: 0.8790 - val_loss: 0.6716 - val_accuracy: 0.7380
Epoch 6/100
10/10 [==============================] - 33s 4s/step - loss: 0.2752 - accuracy: 0.9120 - val_loss: 0.9040 - val_accuracy: 0.6720
Epoch 7/100
10/10 [==============================] - 41s 4s/step - loss: 0.2664 - accuracy: 0.9081 - val_loss: 0.6009 - val_accuracy: 0.7580
Epoch 8/100
10/10 [==============================] - 29s 3s/step - loss: 0.2736 - accuracy: 0.9090 - val_loss: 0.8423 - val_accuracy: 0.6790
Epoch 9/100
10/10 [======

In [6]:
# Load TF Keras model
import tensorflow as tf
model = tf.keras.models.load_model("MobileNet_RTFD.keras")

# Accuracy test with complete dataset
import os
import cv2
import progressbar
import numpy as np

def classify(image):
    # Pre-processing
    image = cv2.resize(image,(224,224))
    image = np.array(np.expand_dims(image,0), dtype=np.float32)
    image = image / 255.
    output = model.predict(image, verbose=0)
    return np.argmax(output, axis=1)[0]

def test_model(folder, classes, length):
    class_idx = 0
    curr_count = 0
    correct_count = 0
    with progressbar.ProgressBar(max_value=length) as bar:
        for c in classes:
            for f in os.listdir(os.path.join(folder, c)):
                bar.update(curr_count)
                image = cv2.imread(os.path.join(folder, c, f))
                out = classify(image)

                if out == class_idx: correct_count += 1
                curr_count += 1
            class_idx += 1

    print(f"Total images           : {curr_count}")
    print(f"Correct predictions    : {correct_count}")
    print(f"Incorrect predictions  : {curr_count-correct_count}")
    print(f"Accuracy               : {correct_count/curr_count*100} %")

# TF Keras
classes = list(train_generator.class_indices.keys())
test_model('Dataset_Pre', classes, 5153)

100% (5153 of 5153) |####################| Elapsed Time: 0:06:18 Time:  0:06:18


Total images           : 5153
Correct predictions    : 4611
Incorrect predictions  : 542
Accuracy               : 89.48185522996313 %
